In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

/home/mathieu/Prose/Mathieu/Benter-Project


In [2]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from winning_validation import errors
from winning_horse_models import sklearn
from winning_horse_models.logistic_regression import LogisticRegressionModel
from winning_horse_models.xgboost import XGBoostWinningModel
from winning_horse_models.lgbm import LGBMWinningModel
from training_procedures import sequential_training, flattened_training
from constants import Sources
from utils import preprocess

from database.setup import create_sqlalchemy_session
from models.race import Race
from models.runner import Runner
tqdm.pandas()

/home/mathieu/.pyenv/versions/3.7.9/envs/benter-project_venv/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
SOURCE = Sources.UNIBET
N_FEATURES = preprocess.get_n_preprocessed_feature_columns(source=SOURCE)

In [4]:
%%capture --no-stdout
%%time
sequential_sgd_regression = LogisticRegressionModel(source=SOURCE, n_features=N_FEATURES)
sequential_sgd_regression, training_history =sequential_training.train_on_each_horse_with_epochs(source=SOURCE, winning_model=sequential_sgd_regression, n_epochs=1, verbose=True)

sequential_sgd_regression.save_model()

Epoch 0

No val data for 2
1/1 [==============================] - 0s 436ms/step - loss: 1.1953 - categorical_accuracy: 0.0000e+00 - categorical_crossentropy: 1.1953
Training for 2 horses (2 races, val 0 races): loss per horse: 0.598, val loss per horse: nan Train Accuracy: 0.0%, Val Accuracy: nan%

1/1 [==============================] - 1s 829ms/step - loss: 1.5124 - categorical_accuracy: 0.3704 - categorical_crossentropy: 1.5124 - val_loss: 0.9016 - val_categorical_accuracy: 0.5000 - val_categorical_crossentropy: 0.9016
Training for 3 horses (27 races, val 4 races): loss per horse: 0.504, val loss per horse: 0.301 Train Accuracy: 37.0%, Val Accuracy: 50.0%

11/11 [==============================] - 0s 21ms/step - loss: 1.3390 - categorical_accuracy: 0.4056 - categorical_crossentropy: 1.3390 - val_loss: 1.4916 - val_categorical_accuracy: 0.3556 - val_categorical_crossentropy: 1.4916
Training for 4 horses (322 races, val 45 races): loss per horse: 0.335, val loss per horse: 0.373 Train A

In [4]:
sequential_sgd_regression = LogisticRegressionModel.load_model()

In [6]:
%%capture --no-stdout
exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=sequential_sgd_regression, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=sequential_sgd_regression, 
                                                            validation_method=errors.kappa_cohen_like)

(0.21928934010152284, 0.09235592714243057, 0.32284263959390863)


In [7]:
print(errors.compute_overall_average(kappa_cohen_1))

(0.14307289600698547, 0.0032002128220759807, 0.2571769690533504)


In [8]:
# Pretrain

In [4]:
%%capture --no-stdout
pretrained_sequential_sgd_regression = LogisticRegressionModel(source=SOURCE,n_features=N_FEATURES)
pretrained_sequential_sgd_regression, training_history =sequential_training.pretrain_on_each_subraces(source=SOURCE,
                                                                                                      winning_model=pretrained_sequential_sgd_regression, 
                                                                                                      n_permutations=10, 
                                                                                                      verbose=True)


exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=pretrained_sequential_sgd_regression, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=pretrained_sequential_sgd_regression, 
                                                            validation_method=errors.kappa_cohen_like)
print(errors.compute_overall_average(kappa_cohen_1))

Pretraining for 2 horses
Computing Subraces of 2 horses with 3-horse races
Computing Subraces of 2 horses with 4-horse races
Computing Subraces of 2 horses with 5-horse races
Computing Subraces of 2 horses with 6-horse races
Computing Subraces of 2 horses with 7-horse races
Computing Subraces of 2 horses with 8-horse races
Computing Subraces of 2 horses with 9-horse races
Computing Subraces of 2 horses with 10-horse races
Computing Subraces of 2 horses with 11-horse races


KeyboardInterrupt: 

In [5]:
%%capture --no-stdout

pretrained_sequential_sgd_regression, training_history =sequential_training.train_on_each_horse_with_epochs(source=SOURCE, winning_model=pretrained_sequential_sgd_regression, n_epochs=1, verbose=True)

Epoch 0

No val data for 2
1/1 [==============================] - 0s 349ms/step - loss: 1.1085 - categorical_accuracy: 0.0000e+00 - categorical_crossentropy: 1.1085
Training for 2 horses (2 races, val 0 races): loss per horse: 0.554, val loss per horse: nan Train Accuracy: 0.0%, Val Accuracy: nan%

1/1 [==============================] - 0s 492ms/step - loss: 1.6767 - categorical_accuracy: 0.4074 - categorical_crossentropy: 1.6767 - val_loss: 0.6269 - val_categorical_accuracy: 1.0000 - val_categorical_crossentropy: 0.6269
Training for 3 horses (27 races, val 4 races): loss per horse: 0.559, val loss per horse: 0.209 Train Accuracy: 40.7%, Val Accuracy: 100.0%

11/11 [==============================] - 0s 18ms/step - loss: 1.4476 - categorical_accuracy: 0.4200 - categorical_crossentropy: 1.4476 - val_loss: 1.5711 - val_categorical_accuracy: 0.3333 - val_categorical_crossentropy: 1.5711
Training for 4 horses (322 races, val 45 races): loss per horse: 0.363, val loss per horse: 0.393 Train 

KeyboardInterrupt: 

In [6]:
%%capture --no-stdout
exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=pretrained_sequential_sgd_regression, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=pretrained_sequential_sgd_regression, 
                                                            validation_method=errors.kappa_cohen_like)
print(errors.compute_overall_average(kappa_cohen_1))

KeyboardInterrupt: 

In [7]:
pretrained_sequential_sgd_regression.save_model(prefix="pretrained_")

In [ ]:
%%time

xgboost_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=XGBoostWinningModel(), verbose=True)
xgboost_winning_model.save_model()

In [ ]:
%%time

knn_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=KNNModel(), verbose=True)
knn_winning_model.save_model()

In [ ]:
%%time
for SklearnModel in [
     #sklearn.DecisionTreeModel, sklearn.SVCModel, #sklearn.KNNModel,
                     #sklearn.RandomForestModel, 
    #sklearn.AdaBoostModel,
                     # sklearn.GradientBoostingModel, sklearn.GaussianNBModel, sklearn.LDAModel,
                     sklearn.SGDModel
                    ]:
    print(SklearnModel.__name__)
    sklearn_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=SklearnModel(), verbose=True)
    sklearn_winning_model.save_model()
    print()

In [ ]:
%%time
sklearn_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=sklearn.LogisticRegressionModel(), verbose=True)

In [ ]:
sklearn_winning_model.save_model()

In [1]:
# APPENDIX

In [5]:
source, on_split=SOURCE, "train"

In [6]:
n_horses=10
y_format = "first_position"

In [7]:
rh_df = import_data.get_split_date(source=source, on_split=on_split)

x = []
y = []
race_dfs = []
for _, race_df in rh_df[rh_df["n_horses"] == n_horses].groupby("race_id"):
    break

In [8]:
import_data.extract_x_y(
            race_df=race_df,
            x_format="sequential_per_horse",
            y_format=y_format,
            source=source,
        )

/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:236: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df[feature_col] = features_df[feature_col].fillna(feature_value)
/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standard_scaler_parameters[feature_col]["mean"]
/home/mathieu/Prose/Mathieu/Benter-Project/utils/preprocess.py:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

AttributeError: 'BooleanArray' object has no attribute 'sum'

In [9]:
y_race = (race_df["horse_place"] == 1).values

In [12]:
y_race.value

AttributeError: 'BooleanArray' object has no attribute 'value'

In [27]:
x_race = preprocess.preprocess(race_horse_df=race_df, source=source)

In [32]:
pd.isnull(x_race).all().any()

False

In [33]:
race_df["horse_place"].isna().all() 

False

In [34]:
y_race = (race_df["horse_place"] == 1).values

In [35]:
y_race

array([False, False, False, False, False, False, False, False, False,
       False])

In [40]:
rh_df['race_id']

runner_id
2208    174
2227    176
1842    141
2272    178
2266    178
       ... 
1392    107
1393    107
1394    107
1430    111
2180    172
Name: race_id, Length: 2199892, dtype: int64

In [42]:
rh_df['horse_place'].dtype

dtype('O')

In [61]:
import datetime as dt
import numpy as np

In [68]:
pd.api.types.is_datetime64tz_dtype(rh_df.dtypes["race_datetime"])

True

In [52]:
rh_df.dtypes["horse_id"] == int

True

In [62]:
np.datetime64()

numpy.datetime64

In [65]:
rh_df.dtypes["race_datetime"]

datetime64[ns, Europe/Paris]

In [67]:
pd.api.types.is_datetime64tz_dtype()

<function pandas.core.dtypes.common.is_datetime64tz_dtype(arr_or_dtype) -> bool>

In [16]:
from utils.import_data import *

In [15]:
df['horse_place']=df['horse_place'].astype("Int64")

NameError: name 'df' is not defined

In [19]:
df['horse_place']

runner_id
2208       4
2227       6
1842    <NA>
2272    <NA>
2266    <NA>
        ... 
1392       2
1393       1
1394       4
1430    <NA>
2180    <NA>
Name: horse_place, Length: 2839231, dtype: Int64

In [17]:
df=pd.read_parquet(os.path.join(DATA_DIR, "unibet_data_with_features.parquet"))

In [22]:
df['horse_place']=df['horse_place'].astype(float)

In [23]:
df['horse_place']

runner_id
2208    4.0
2227    6.0
1842    NaN
2272    NaN
2266    NaN
       ... 
1392    2.0
1393    1.0
1394    4.0
1430    NaN
2180    NaN
Name: horse_place, Length: 2839231, dtype: float64

In [24]:
df.to_parquet(os.path.join(DATA_DIR, "unibet_data_with_features.parquet"))

In [25]:
memory.clear()